# LDA主题模型学习总结

`本篇博客是《LDA漫游指南》和《LDA数学八卦》的学习笔记。`

## 目录

- [简介](#简介)
    - [LDA算法输入与输出](#LDA算法输入与输出)
- [前置知识](#前置知识)
    - [gamma函数](#gamma函数)
    - [二项分布](#二项分布)
    - [Beta分布](#Beta分布)
    - [多项分布](#多项分布)
    - [Dirichlet分布](#Dirichlet分布)
    - [共轭先验分布](#共轭先验分布)
    - [MCMC](#MCMC)
- [LDA推导](#LDA推导)
    - [贝叶斯unigram](#贝叶斯unigram)
    - [LDA模型的标准生成过程](#LDA模型的标准生成过程)
    - [数学表示](#数学表示)
- [交给Gibbs Sampling](#交给Gibbs-Sampling)
    
    

## 简介

LDA（Latent Dirichlet Allocation）是一种**非监督**机器学习技术，可以用来识别大规模文档集或语料库中潜在隐藏的主题信息。

LDA假设每个词是由背后的一个潜在隐藏的主题中抽取出来的，对于每篇文档，生成过程如下：
- 1.对于每篇文档，从主题分布中抽取一个主题。
- 2.从上述被抽到的主题所对应的单词分布中抽取一个单词。
- 3.重复上述过程直到遍历文档中的每个单词。

### LDA算法输入与输出
- 输入：分词后的文章集。主题数$K$，超参数：$\alpha$和$\beta$。
- 输出：
    - 1.每篇文章每个词被指定的主题编号。
    - 2.每篇文章的主题概率分布：$\theta$
    - 3.每个主题下的词概率分布：$\phi$
    - 4.词和id的映射表。
    - 5.每个主题$\phi$下


## 前置知识

### gamma函数

所谓的gamma函数就是阶乘的函数形式。

$$\Gamma(x)=\int_0^{+\infty}e^{-t}t^{x-1}dt\;\;\;(x>0)$$

$$\Gamma(n) = (n-1)!$$

### 二项分布

打靶，$n$次中中了$k$次的概率：

$$f(k;n,p)=Pr(X=k)=\binom{n}{k}p^k(1-p)^{n-k}$$

### Beta分布

$X\sim Beta(\alpha, \beta)$

概率密度函数：$$f(x;\alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)}x^{\alpha-1}(1-x)^{\beta-1}\\=\frac{1}{B(\alpha, \beta)}x^{\alpha-1}(1-x)^{\beta-1}$$

期望：$$E(p) = \int_0^1t\cdot Beta(t|\alpha, \beta)dt\\=\frac{\alpha}{\alpha+\beta}$$

### 多项分布

多项分布是二项分布的推广：投$n$次骰子，共有六种结果，概率为$p_i$，$i$点出现$x_i$次的组合概率：

$$f(x_1, ...x_k;n,p_1,...,p_k)=Pr(X_1=x_1\; and\; ... and\; X_k=x_k)\\=\frac{n!}{x_1!...x_k!}p_1^{x_1}...p_k^{x_k}\;\;\;when\;\sum_{i=1}^kx_i = n$$

### Dirichlet分布

$$p\sim D(t|\alpha)$$

概率密度函数：$$f(p_1,..., p_k-1)=\frac{1}{\Delta (\alpha)}\prod_{i=1}^kp_i^{\alpha_i-1}$$

期望：$$E(p) = (\frac{\alpha_1}{\sum_{i=1}^K\alpha_i}, \frac{\alpha_2}{\sum_{i=1}^K\alpha_i}, ..., \frac{\alpha_K}{\sum_{i=1}^K\alpha_i})$$

### 共轭先验分布

贝叶斯公式：$$p(\theta|x) = \frac{p(x|\theta)p(\theta)}{p(x)}$$

即：**后验分布=似然函数×先验分布**

**共轭**：选取一个函数作为似然函数，使得先验分布函数和后验分布函数的形式一致。

- beta分布是二项分布的共轭先验分布，即，二项分布作为似然函数，先验分布是beta分布，后验分布依然是beta分布。
- Dirichlet分布是多项式分布的共轭先验分布，即，多项式布作为似然函数，先验分布是Dirichlet分布，后验分布依然是Dirichlet分布。

### MCMC

参考之前的博客：https://applenob.github.io/1_MCMC.html

## LDA推导

### 贝叶斯unigram

不考虑单词简单顺序，被称为“词袋模型”。

$$P(W) = p(w_1)p(w_2)...p(w_n) = \prod^V_{t=1}p_t^{n_t}\;\;\;\sum^V_{t=1}p_t = 1$$

为什么似然是多项式分布？想象一个巨大的骰子，有$V$个面，每面代表一个词，每个面的概率是$\vec{p}=(p_1, ...p_V)$，产生次数是：$\vec{n} = (n_1, ..., n_V)$，那么生成某篇文章的概率是服从多项式分布的。

贝叶斯学派认为参数也服从某种分布，即，不知道上帝用哪个骰子来生成文档，这个选取骰子的概率，服从Dirichlet分布。

又有：$Dir(\vec{p}|\vec{\alpha}) + MultCount(\vec{n}) = Dir(\vec{p}|\vec{\alpha}+\vec{n})$，综合上面Dirichlet分布的期望，可以得到对于每一个$p_i$，可以如下**估计**：$\tilde p_i = \frac{n_i+\alpha_i}{\sum_{i=1}^V(n_i + \alpha_i)}$。即，每个参数的估计值是其对应事件的先验的伪计数和数据中的计数的和在整体技术中的比例。

进一步，计算出**文本语料的产生概率**是：$p(W|\vec{\alpha}) = \int p(W|\vec{p})p(\vec{p}|\vec{\alpha})d\vec{p}=\frac{\Delta(\vec{n}+\vec{\alpha})}{\Delta \vec{\alpha}}$

![](https://github.com/applenob/machine_learning_basic/raw/master/res/bayes_unigram.png)

用通俗的话说，这是上帝从一个坛子中抽一个骰子，再丢这个骰子，观察结果的过程。

### LDA模型的标准生成过程

LDA相当于两个上面的步骤的结合（两个坛子）。上帝有两个大坛子，第一个坛子装doc-topic骰子，第二个坛子装topic-word骰子：

- 1.选择$\theta_i \sim Dir(\vec{\alpha})$，这里$i\in\{1,2,...,M\}$，$M$代表文章数。每生成一篇文章，从第一个坛子中选一个doc-topic骰子。
- 2.选择$\phi_i \sim Dir(\vec{\beta})$，这里$k \in \{1,2,...,K\}$，$K$代表主题个数。独立地挑了$K$个topic-word骰子。
- 3.对每个单词的位置$W_{i,j}$，这里$j \in \{1,...,N_i\}$，$i \in \{1,...,M\}$
    - 4.选择一个topic主题：$z_{i,j} \sim Multinominal(\theta_i)$。投掷这个doc-topic骰子，得到一个topic编号$z$。
    - 5.选择一个word词：$w_{i,j} \sim Multinominal(\phi_{z_{i,j}})$。投掷topic是$z$的topic-word骰子，得到一个词。
    
![](https://github.com/applenob/machine_learning_basic/raw/master/res/lda.png)


### 数学表示

对每个doc-topic骰子，有：$p(\vec z_m | \vec \alpha) = \frac{\Delta(\vec n_m +\vec  \alpha)}{\Delta \vec \alpha}$

其中：$\vec n_m = (n_m^{(1)}, .., n_m^{(K)})$，$n_m^{(k)}$代表第$m$篇文档中第$k$个topic产生的词的个数。$\vec \alpha$是$K$维的向量。

因为M篇文章生成topic的过程上**相互独立**的，有M个doc-topic骰子的联合概率分布，即**整个语料的topics生成概率**：$$p(\vec z| \vec \alpha) = \prod^M_{m=1}p(\vec z_m|\vec \alpha)\\=\prod^M_{m=1}\frac{\Delta(\vec n_m + \vec \alpha)}{\Delta \vec \alpha}$$

对每topic-word骰子，有：$p(\vec w_k|\vec \beta) = \frac{\Delta(\vec n_k + \vec \beta)}{\Delta \vec \beta}$

其中：$\vec n_k = (n_k^{(1)}, .., n_k^{(V)})$，$n_k^{(v)}$代表第$k$个topic产生的词中，第$v$个word产生的词的个数。$\vec \beta
$是$V$维的向量。

因为K个topic生成word的过程也是**相互独立**的，有K个topic-word骰子的联合概率分布，即**整个语料中words生成的概率**：$$p(\vec w|\vec z, \vec \beta)\\=\prod_{k=1}^Kp(\vec w_{(k)}|\vec z_{(k)}, \vec \beta)\\=\prod_{k=1}^K\frac{\Delta(\vec n_k + \vec \beta)}{\Delta \vec \beta}$$

联合上面两个联合概率分布，得到**整个语料中words生成的概率**和**整个语料的topics生成概率**的**联合概率分布**：

$$p(\vec w, \vec z| \vec \alpha, \vec \beta)\\=p(\vec w|\vec z, \vec \beta)p(\vec z| \vec \alpha)\\=\prod_{k=1}^K\frac{\Delta(\vec n_k + \vec \beta)}{\Delta \vec \beta}\prod^M_{m=1}\frac{\Delta(\vec n_m + \vec \alpha)}{\Delta \vec \alpha}$$

## 交给Gibbs Sampling

Gibbs Smapling建议先回顾下之前的[博客文章](https://applenob.github.io/1_MCMC.html#Gibbs-Sampling)。

有了联合分布$p(\vec w, \vec z)$，可以使用Gibbs Sampling了。

语料库中第$i$个词对应的topic记为$z_i$，其中$i=(m,n)$是一个二维下标，对应第$m$篇文档的第$n$个词。$-i$表示去除下标$i$的词。

我们要采样的分布是$p(\vec z| \vec w)$，根据Gibbs Sampling的要求，我们要知道**完全条件概率(full conditionals)**，这里即：$p(z_i=k|\vec z_{-i}, \vec w)$。设观测到的词$w_i=t$，根据贝叶斯法则，有：$p(z_i=k|\vec z_{-i}, \vec w)\propto p(z_i=k, w_i=t|\vec z_{-i}, \vec w_{-i})$

**完整推导**：
![](https://github.com/applenob/machine_learning_basic/raw/master/res/lda_gibbs.png)

**最终的Gibbs Smapling公式**：

$$p(z_i=k|\vec z_{-i}, \vec w)\propto \frac{n^{k}_{m,-i}+\alpha_k}{\sum_{k=1}^K(n^{k}_{m,-i}+\alpha_k)} \cdot \frac{n^{t}_{k,-i}+\beta_t}{\sum_{t=1}^V(n^{t}_{k,-i}+\beta_t)}$$

右边是$p(topic|doc)\cdot p(word|topic)$，这个概率其实是$doc\leftarrow topic \leftarrow word$的路径概率。

![]()